In [1]:
import re, collections
import nltk
from HolbrookCorpus import HolbrookCorpus

In [2]:
# From norvig.com/spell-correct.html

def words(text): 
    return re.findall('[a-z]+', text.lower()) 

def train(features):
    model = collections.defaultdict(lambda: 1)
    for f in features:
        model[f] += 1
    return model

In [3]:
trainPath = '../data/holbrook-tagged-train.dat'
trainingCorpus = HolbrookCorpus(trainPath)

In [4]:
error_corpus = [s.getErrorSentence() for s in trainingCorpus.corpus]
error_corpus[3:5]

[['<s>', 'my', 'dad', 'works', 'at', 'melton', '</s>'],
 ['<s>', 'my', 'siter', 'go', 'to', 'tonbury', '</s>']]

In [5]:
correct_corpus = [s.getCorrectSentence() for s in trainingCorpus.corpus]
correct_corpus[3:5]

[['<s>', 'my', 'dad', 'works', 'at', 'melton', '</s>'],
 ['<s>', 'my', 'sister', 'goes', 'to', 'tonbury', '</s>']]

In [6]:
flattened_corpus = nltk.flatten(correct_corpus)
NWORDS = train(flattened_corpus)

In [7]:
# From norvig.com/spell-correct.html

alphabet = 'abcdefghijklmnopqrstuvwxyz'

def edits1(word):
    splits     = [(word[:i], word[i:]) for i in range(len(word) + 1)]
    deletes    = [a + b[1:] for a, b in splits if b]
    transposes = [a + b[1] + b[0] + b[2:] for a, b in splits if len(b)>1]
    replaces   = [a + c + b[1:] for a, b in splits for c in alphabet if b]
    inserts    = [a + c + b     for a, b in splits for c in alphabet]
    return set(deletes + transposes + replaces + inserts)

def known_edits2(word):
    return set(e2 for e1 in edits1(word) for e2 in edits1(e1) if e2 in NWORDS)

def known(words): 
    return set(w for w in words if w in NWORDS)

def correct(word):
    candidates = known([word]) or known(edits1(word)) or known_edits2(word) or [word]
    return max(candidates, key=NWORDS.get)

In [8]:
correct('speling')  # notice how this differs from Norvig, why?

'seeing'

In [9]:
cfd = nltk.ConditionalFreqDist(nltk.bigrams(flattened_corpus))

In [10]:
cfd['be']

FreqDist({'a': 4, 'nice': 2, 'careful': 2, 'killed': 2, 'seeing': 1, 'doing': 1, 'back': 1, 'two': 1, 'in': 1, 'rough': 1, ...})

In [11]:
# from https://sites.google.com/site/gothnlp/exercises/jurafsky-martin/solutions
vocabulary = set(flattened_corpus)  
cpd_laplace  = nltk.ConditionalProbDist(cfd, nltk.LaplaceProbDist, bins=len(vocabulary))

In [12]:
cpd_laplace['be'].prob('seeing')

0.0011799410029498525

In [13]:
cpd_laplace['be'].prob('speling')

0.0005899705014749262